In [47]:
import numpy as np
import cv2
from imutils.perspective import four_point_transform
import imutils

In [355]:
image = cv2.imread("../samples/n1.png")
image = imutils.resize(image, height=1200)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 75, 200)

# cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnts = imutils.grab_contours(cnts)
# docCnt = None

# if len(cnts) > 0:
# 	# sort the contours according to their size in
# 	# descending order
# 	cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
# 	# loop over the sorted contours
# 	for c in cnts:
# 		# approximate the contour
# 		peri = cv2.arcLength(c, True)
# 		approx = cv2.approxPolyDP(c, 0.02 * peri, True)
# 		# if our approximated contour has four points,
# 		# then we can assume we have found the paper
# 		if len(approx) == 4:
# 			docCnt = approx
# 			break
            
# gray = four_point_transform(gray, docCnt.reshape(4, 2))

In [356]:
cv2.imshow("image", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [361]:
# image_obj = cv2.imread('../samples/foto.jpg')
# image_obj = imutils.resize(image_obj, height=1200)

# gray = cv2.cvtColor(image_obj, cv2.COLOR_BGR2GRAY)

image_obj = image.copy()

# blur = cv2.GaussianBlur(gray, (5, 5), 0)

kernel = np.ones((4, 4), np.uint8)
dilation = cv2.dilate(blur, kernel, iterations=1)

thresh = cv2.adaptiveThreshold(dilation, 255, 1, 1, 11, 2)

# cv2.imshow("image", thresh)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Now finding Contours         ###################
contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
print(len(contours))
coordinates = []
for cnt in contours:
        # [point_x, point_y, width, height] = cv2.boundingRect(cnt)
    approx = cv2.approxPolyDP(cnt, 0.04 * cv2.arcLength(cnt, True), True)
    if len(approx) == 3:
        coordinates.append(approx)
# cv2.drawContours(image_obj, [cnt], 0, (0, 0, 255), 3)

print(len(coordinates))
cv2.imwrite("result.png", image_obj)

1640
32


True

In [362]:
coordinates.sort(key=lambda x: cv2.contourArea(x), reverse=True)
triangles = coordinates[:4]
triangles, boxes = imutils.contours.sort_contours(triangles, method='left-to-right')

In [363]:
for i, c in enumerate(triangles):
    sorted_image = imutils.contours.label_contour(image_obj, c, i, color=(0, 0, 255))
cv2.imwrite("result.png", sorted_image)

True

In [360]:
triangles[1][0]

array([[ 49, 999]], dtype=int32)

In [338]:
trilist = []
for triangle in triangles:
    tripoints = []
    for point in triangle:
        tripoints.append((point[0][0], point[0][1]))
    dtype = [('x', int), ('y', int)]
    a = np.array(tripoints, dtype=dtype)
    trilist.append(a)
triangles = np.array(trilist)
triangles

array([[( 49,   40), ( 34,   42), ( 33,   58)],
       [( 49,  999), ( 52, 1016), ( 68, 1017)],
       [(681,   29), (700,   46), (697,   30)],
       [(719,  989), (702, 1007), (719, 1005)]],
      dtype=[('x', '<i8'), ('y', '<i8')])

In [339]:
triangles[0]

array([(49, 40), (34, 42), (33, 58)], dtype=[('x', '<i8'), ('y', '<i8')])

In [331]:
p00 = triangles[0]
p00 = p00[np.lexsort((p00['y'], p00['x']))][0]

p10 = triangles[1]
p10 = p10[np.lexsort((p10['x'], -p10['x']))][0]

In [305]:
p00x = min(triangles[0], key=lambda x: x[0])[0]
p00y = min(triangles[0], key=lambda x: x[1])[1]
p00 = (p00x, p00y)

p10x = min(triangles[1], key=lambda x: x[0])[0]
p10y = max(triangles[1], key=lambda x: x[1])[1]
p10 = (p10x, p10y)

p01x = max(triangles[2], key=lambda x: x[0])[0]
p01y = min(triangles[2], key=lambda x: x[1])[1]
p01 = (p01x, p01y)

p11x = max(triangles[3], key=lambda x: x[0])[0]
p11y = max(triangles[3], key=lambda x: x[1])[1]
p11 = (p11x, p11y)

In [306]:
p0x, p0y

(74, 84)

In [307]:
p00, p10, p01, p11

((33, 40), (49, 1017), (700, 29), (719, 1007))

In [308]:
warped = imutils.perspective.four_point_transform(gray.copy(), np.array([p00, p01, p10, p11]))
cv2.imwrite("warped.png", warped)

True

In [309]:
cv2.imshow("image", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()